# Part A: 
Get the program earthquake,py to run with your MongoDB. Don't forget to start the MongoDB server before running.

Note that the earthquake_json program collects data and puts it in the database. In practice, you would write a separate program that would then be able to explore or analyze that data. But in order to keep the exercise short, we are going to add another section to the same program to access the data.


Submit your updated program and its output in a single text file.

In [1]:
import urllib.request
import json
import pymongo


# get the bbc rss feed of news stories and connect to it
earthquake_url = "http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson"

try:
	response = urllib.request.urlopen(earthquake_url)
except urllib.error.URLError as e:
    if hasattr(e, 'reason'):
        print('We failed to reach a server.')
        print('Reason: ', e.reason)
    elif hasattr(e, 'code'):
        print('The server couldn\'t fulfill the request.')
        print('Error code: ', e.code)
else:
	# the url request was successful - convert the response to a string
	json_string = response.read().decode('utf-8')

	# the json package loads() converts the string to python dictionaries and lists
	eq_json = json.loads(json_string)
	
	# from the json dictionary we get the title to print
	title = eq_json['metadata']['title']
	print('Collected data from', title)
	#  and we get the list of earthquakes
	quakelist = eq_json['features']

	# Connection to Mongo DB
	try:
	    client=pymongo.MongoClient('localhost', 27017)
	    print ("Connected successfully!!!")
	except pymongo.errors.ConnectionFailure as e:
	   print ("Could not connect to MongoDB: %s" % e )
	else:

		# use database named usgs or create it if not there already
		eqdb = client.usgs
		# create collection named earthquakes or create it if not there already
		quakecoll = eqdb.earthquakes
		# add all the earthquakes to the list
		quakecoll.insert_many(quakelist)
		print("Added", len(quakelist), "to earthquakes collection in usgs database")
		# close the database connection
		client.close()



Collected data from USGS Significant Earthquakes, Past Month
Connected successfully!!!
Added 16 to earthquakes collection in usgs database


# Part B: 
Add to the program to get back the earthquakes from the database and print the 'place' and 'time' for each quake. That is, you should add code to the program that after it adds the earthquakes to Mongo, it should get them back out with find() and print those two items for each earthquake.



# Solution

In [8]:
''' 
	This program accesses the USGS earthquake data feed. This URL is for
       significant quakes in the last 30 days.
	It then saves the earthquakes to a database named usgs and a collection called earthquakes.
	The mongod server must be running for this program to work!
'''

import urllib.request
import json
import pymongo


# get the bbc rss feed of news stories and connect to it
earthquake_url = "http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson"

try:
	response = urllib.request.urlopen(earthquake_url)
except urllib.error.URLError as e:
    if hasattr(e, 'reason'):
        print('We failed to reach a server.')
        print('Reason: ', e.reason)
    elif hasattr(e, 'code'):
        print('The server couldn\'t fulfill the request.')
        print('Error code: ', e.code)
else:
	# the url request was successful - convert the response to a string
	json_string = response.read().decode('utf-8')

	# the json package loads() converts the string to python dictionaries and lists
	eq_json = json.loads(json_string)
	
	# from the json dictionary we get the title to print
	title = eq_json['metadata']['title']
	print('Collected data from', title)
	#  and we get the list of earthquakes
	quakelist = eq_json['features']

	# Connection to Mongo DB
	try:
	    client=pymongo.MongoClient('localhost', 27017)
	    print ("Connected successfully!!!")
	except pymongo.errors.ConnectionFailure as e:
	   print ("Could not connect to MongoDB: %s" % e )
	else:

		# use database named usgs or create it if not there already
		eqdb = client.usgs
		# create collection named earthquakes or create it if not there already
		quakecoll = eqdb.earthquakes
		# add all the earthquakes to the list
		quakecoll.insert_many(quakelist)
		print("Added", len(quakelist), "to earthquakes collection in usgs database")
		
	# Grab back the collection of earthquakes from MongoDB
		quake_docs = quakecoll.find()
		# Loop through and print out place and time of the earthquakes
		for doc in quake_docs:
			place = doc["properties"]["place"]
			# grab unix timestamp in milliseconds
			unix_time_mil = doc["properties"]["time"]
			# convert to unix in seconds
			unix_time = unix_time_mil / 1000
			# use datatime to convert to human readible time format, goes down to milliseconds
			# print the place and time, with a label to show the time is in UTC
			print('Place: ',place,'        Time: ',unix_time)
# close the database connection
		client.close()



Collected data from USGS Significant Earthquakes, Past Month
Connected successfully!!!
Added 16 to earthquakes collection in usgs database
Place:  41km NW of Platanos, Greece         Time:  1574839422.552
Place:  6km ESE of Blagaj, Bosnia and Herzegovina         Time:  1574759966.08
Place:  16km WSW of Mamurras, Albania         Time:  1574736852.594
Place:  85km ESE of Adak, Alaska         Time:  1574556842.199
Place:  31km ESE of Chaloem Phra Kiat, Thailand         Time:  1574293844.205
Place:  267km NW of Ozernovskiy, Russia         Time:  1574238367.542
Place:  111km SW of Puerto Madero, Mexico         Time:  1574224025.921
Place:  2km N of Kadingilan, Philippines         Time:  1574083330.217
Place:  138km E of Bitung, Indonesia         Time:  1573748260.457
Place:  26km W of Pepeekeo, Hawaii         Time:  1573490152.79
Place:  167km SSW of Ndoi Island, Fiji         Time:  1573209884.876
Place:  57km NE of Hashtrud, Iran         Time:  1573166825.428
Place:  6km W of Ventura, CA  